# Exercise 2: Umbrella sampling

In this exercise, we will perform umbrella sampling simulations for the NaCl system, which is composed of the following 3 steps that we will elaborate later:
- A pulling simulation (a.k.a steered MD) 
- Generation of the initial configuraitons for the production runs
- Production runs. 


This exercise should be done in an interactive session on Bridges-2, which 32 CPU cores requested. Note that here we are running the simulations in the NVT ensemble, which makes pulling more straightforward.

## 1. Pulling simulation

In [1]:
%%bash
mkdir pull

The purpose of a pulling simulation is to generate initial configurations with different CV values for the subsequent production simulations. In a pulling simulation, we need to divide the system of interest (NaCl) into two parts 

- The immobile group: The part of the system that is immobilized by a **position restraint**. In this exercise, we choose to fix the sodium ion, but choosing to fix the chloride ion should not make a difference in the resulting free energy profile. 

<center> 
➡️ This can be done by modifying the TOP file.
</center>

- The pull group: The part of the system we want to pull away from the immobile group using the **pull code**. Here, we choose the chloride ion as the pull group. 

<center> 
➡️ This can be done by modifying the MDP file.
</center>

## 1-1. Modification of the `top` file

In [2]:
%%bash
cat ../Inputs/NaCl/NaCl.top  # the original top file we used

#include "oplsaa.ff/forcefield.itp"
#include "oplsaa.ff/tip3p.itp"

[ moleculetype ]
; molname   nrexcl
CL      1

[ atoms ]
; id    at type     res nr  residu name at name  cg nr  charge   mass
1       opls_401    1       CL          CL       1      -1       35.45300

[ moleculetype ]
; molname   nrexcl
NA          1

[ atoms ]
; id    at type     res nr  residu name at name  cg nr  charge   mass
1       opls_407    1       NA          NA       1      1        22.98977

[ System ]
NaCl in water

[ Molecules ]
SOL         107
NA               1
CL               1


To make the topology aware of the position restraint, we need to add the following right after the `[ atoms ]` directive of NA.

In [3]:
%%bash 
echo '
#include "oplsaa.ff/forcefield.itp"
#include "oplsaa.ff/tip3p.itp"

[ moleculetype ]
; molname   nrexcl
CL      1

[ atoms ]
; id    at type     res nr  residu name at name  cg nr  charge   mass
1       opls_401    1       CL          CL       1      -1       35.45300

[ moleculetype ]
; molname   nrexcl
NA          1

[ atoms ]
; id    at type     res nr  residu name at name  cg nr  charge   mass
1       opls_407    1       NA          NA       1      1        22.98977

; position restraints for Na
# ifdef POSRES_NA
# include "Na_posres.itp"
# endif

[ System ]
NaCl in water

[ Molecules ]
SOL         107
NA               1
CL               1
' > pull/NaCl_US.top

```
; position restraints for Na
# ifdef POSRES_NA
# include "Na_posres.itp"
# endif
```

The added lines mean that **if the variable `POSRES_NA` is defined (in the `mdp` file), the position restraint defined in the file `Na_posres.itp` will be activated**. 

Clearly, now we don't have the file `Na_posres.itp`, but we will worry about that later.

To get the `itp` file to apply the position restraint, we need to run the GROMACS `genrestr` command, which requires a `gro` file.

In [4]:
%%bash
bash /ocean/projects/see220002p/shared/icomse-setup.sh  # load in necessary modules to enable GROMACS
cp ../Inputs/NaCl/NaCl.gro pull/.    # copy over the gro file
echo NA | mpirun -np 1 gmx_mpi genrestr -f pull/NaCl.gro -o pull/Na_posres.itp

              :-) GROMACS - gmx genrestr, 2020.6-plumed-2.8.0 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders

Select group to position restrain


Group     0 (         System) has   323 elements
Group     1 (          Water) has   321 elements
Group     2 (            SOL) has   321 elements
Group     3 (      non-Water) has     2 elements
Group     4 (            Ion) has     2 elements
Group     5 (             NA) has     1 elements
Group     6 (             CL) has     1 elements
Group     7 ( Water_and_ions) has   323 elements


Selected 5: 'NA'


Select a group: 
GROMACS reminds you: "Measuring programming progress by lines of code is like measuring aircraft building progress by weight." (Bill Gates)



In [5]:
%%bash
cat pull/Na_posres.itp

; position restraints for NA of NaCl in water t= 200.00000 step= 100000

[ position_restraints ]
;  i funct       fcx        fcy        fcz
 322    1       1000       1000       1000


Importantly, the file `Na_posres.itp` needs to be modified because it was generated based on `NaCl.gro`, where the sodium ion had an index of 322. However, in `NaCl.top`, the lines mentioning the restraint were inserted before the `[ Molecules ]` directive, so the `idp` file is not aware of the whole system of 323 atoms but only the atom in the last `[ molecules ]` directive, which is the NA atom and has an index of 1. Therefore, we should use the index of 1 in the `itp` file as well. 

In [6]:
%%bash
sed -i -e "s/ 322    1       1000       1000       1000/ 1    1       1000       1000       1000/g" pull/Na_posres.itp
cat pull/Na_posres.itp

; position restraints for NA of NaCl in water t= 200.00000 step= 100000

[ position_restraints ]
;  i funct       fcx        fcy        fcz
 1    1       1000       1000       1000


## 1-2. Modification of the `mdp` file

In [7]:
%%bash
cat ../Inputs/NaCl/MD-NVT.mdp  # the original MDP file

integrator = md 
dt = 0.002
nsteps = 250000
cutoff-scheme = Verlet
coulombtype = PME
rlist = 0.6
rcoulomb = 0.6
rvdw = 0.6
constraints = h-bonds
tcoupl =  V-rescale
ref_t = 300
tau-t = 1.0
tc-grps = System
gen-vel = yes
gen-temp = 300
gen-seed = -1 
DispCorr = AllEnerPres
nstxout-compressed = 50
nstxout = 50000
nstvout = 50000


First, we need to define `POSRES_NA` at the top of the MDP file so the position restraint is activated. This is done by adding the following line at the top of the `mdp` file:
```
define=-DPOSRES_NA
```

In [8]:
%%bash
echo '
define=-DPOSRES_NA
integrator = md 
dt = 0.002
nsteps = 125000
cutoff-scheme = Verlet
coulombtype = PME
rlist = 0.6
rcoulomb = 0.6
rvdw = 0.6
constraints = h-bonds
tcoupl =  V-rescale
ref_t = 300
tau-t = 1.0
tc-grps = System
gen-vel = yes
gen-temp = 300
gen-seed = -1 
compressibility = 4.5e-5
nstxout-compressed = 50
nstxout = 50000
nstvout = 50000
' > pull/NaCl_pull.mdp
# Here we create an mdp file starting with `define=-DPOSRES_NA`

Then, to pull the chloride ion away, we need to use the pull code in the `mdp` file.

In [9]:
%%bash
echo '
; Pull code
pull = yes
pull-ncoords = 1                  ; Here we only have 1 CV.
pull-ngroups = 2                  ; We have 2 groups (one immobile/reference group and one pull group).
pull-group1-name = NA             ; index 1
pull-group2-name = CL             ; index 2
pull-coord1-groups = 1 2          ; groups with indices 1 (NA) and 2 (CL) are involved
pull-coord1-type = umbrella       ; harmonic potential
pull-coord1-geometry = distance   ; simple distance increase
pull-coord1-dim = Y Y Y           ; We allow pulling from any directions.
pull-coord1-start = yes           ; Define initial COM distance > 0
pull-coord1-rate = 0.0010         ; 0.0010 nm/ps -> the pull distance in the 250 ps-simulation is 0.25 nm.
pull-coord1-k = 1000              ; units: kJ/mol/nm^2
' >> pull/NaCl_pull.mdp

Here are more explanations for some of the parameters:

- `pull_coord1_rate`: The rate at which the imaginary spring attached between the ions is elongated. Importantly, the pull distance must not 0.7428 nm, which is half of the smallest dimension of the box. This is to prevent the pull group from interacting with the periodic image of the system. Given that the typical ion-pair distance of NaCl is around 0.265 nm, a total pull distance of 0.3 nm should be safe.  

## 1-3. Running a pulling simulation

Now, we can finally run the pulling simulation. Notably, the `-r` option is necessary to be included for the restraint. 

In [10]:
%%time
%%bash
mpirun -np 1 gmx_mpi grompp -f pull/NaCl_pull.mdp -c pull/NaCl.gro -r pull/NaCl.gro -p pull/NaCl_US.top -o pull/pull.tpr -po pull/mdout.mdp -maxwarn 1

               :-) GROMACS - gmx grompp, 2020.6-plumed-2.8.0 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:

turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...


Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to -1610686977
Velocities were taken from a Maxwell distribution at 300 K


Analysing residue names:
There are:   107      Water residues
There are:     2        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K


Pull group 1 'NA' has 1 atoms
Pull group 2 'CL' has 1 atoms
Number of degrees of freedom in T-Coupling group System is 645.00

NOTE 1 [file pull/NaCl_pull.mdp]:
  Removing center of mass motion in the presence of position restraints
  might cause artifacts. When you are using position restraints to
  equilibrate a macro-molecule, the artifacts are usually negligible.



Calculated rlist for 1x1 atom pair-list as 0.638 nm, buffer size 0.038 nm
Set rlist, assuming 4x4 atom pair-list, to 0.607 nm, buffer size 0.007 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 14x14x14, spacing 0.106 0.106 0.106


Pull group  natoms  pbc atom  distance at start  reference at t=0
       1         1         0
       2         1         0       0.280 nm          0.280 nm
Estimate for the relative computational load of the PME mesh part: 0.42

There was 1 note


This run will generate roughly 5 Mb of data



GROMACS reminds you: "All models are wrong, but some are useful." (George Box)



CPU times: user 18.8 ms, sys: 7.62 ms, total: 26.4 ms
Wall time: 327 ms


In [ ]:
%%time
%%bash
mpirun -np 8 gmx_mpi mdrun -deffnm pull/pull -pf pull/pullf.xvg -px pull/pullx.xvg -ntomp 4

                :-) GROMACS - gmx mdrun, 2020.6-plumed-2.8.0 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:

Compared to standard MD simulation, a pulling simulation has two additional output files: `pullf.xvg` and `pullf.xvg`, which document the timeseries of the pulling force and the ion-pair distance. Below we plot both of them. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data_1 = np.transpose(np.loadtxt('pull/pullx.xvg', comments=['@', '#']))
data_2 = np.transpose(np.loadtxt('pull/pullf.xvg', comments=['@', '#']))
time = data_1[0]
dist = data_1[1]
force = data_2[1]

fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(121)
plt.plot(time, dist)
plt.xlabel('Time (ps)')
plt.ylabel('Pulling distance (nm)')
plt.grid()

ax = fig.add_subplot(122)
plt.plot(time, force)
plt.xlabel('Time (ps)')
plt.ylabel('Pulling force (kJ/mol/nm)')
plt.grid()

## 2. Generation of the initial configurations

Given that we have 32 CPU available, having 16 windows should be reasonable, so each production run can be performed with 2 CPU cores, which should be enough as we explored preliminarily in the previous exercise. From the pull distance 

First, we need to decide the number of windows to use.

In [ ]:
import os
centers = np.linspace(min(dist), 0.6, 8)  # the spacing between windows should be around 0.05 nm
diff_list = np.abs([dist - i for i in centers])
diff_idx = [np.argmin(diff_list[i]) for i in range(len(centers))]  # dist[diff_idx]: distances closet to the centers
t_extract = time[diff_idx]  # in ps
print(f'The initial configurations will be extracted from the following time frames (in ps): {t_extract}')
print(f'The ion-pair distances (in nm) of the chosen configurations are: {dist[diff_idx]}')

In [ ]:
for i in range(len(t_extract)):
    os.mkdir(f'sim_{i + 1}')
    os.system(f'echo System | mpirun -np 1 gmx_mpi trjconv -s pull/pull.tpr -f pull/pull.xtc -dump {t_extract[i]} -o sim_{i + 1}/NaCl_{i + 1}.gro')

In [ ]:
%%bash
cp pull/NaCl_pull.mdp NaCl_umbrella.mdp
sed -i -e "s/nsteps = 125000/nsteps = 250000/g" NaCl_umbrella.mdp  # simulation length: 500 ps for each umbrella
sed -i -e "s/pull-coord1-rate = 0.0010         ; 0.0010 nm\/ps -> the pull distance in the 250 ps-simulation is 0.25 nm./pull-coord1-rate = 0/g" NaCl_umbrella.mdp
sed -i -e "s/pull-coord1-k = 1000              ; units: kJ\/mol\/nm^2/pull-coord1-k = 10000/g" NaCl_umbrella.mdp  # simulation length: 500 ps for each umbrella
cat NaCl_umbrella.mdp

In [ ]:
%%bash
n=8  # number of simulations
for i in $(seq 1 $n)
do
    cp pull/NaCl_US.top sim_${i}/.    # Use the same top file as in the pulling simulation
    cp pull/Na_posres.itp sim_${i}/.  # Copy over the itp file for position restraint
    cp NaCl_umbrella.mdp sim_${i}/.   # All simulations use the same NaCl_umbrella.mdp
    cd sim_${i}
    mpirun -np 1 gmx_mpi grompp -f NaCl_umbrella.mdp -c NaCl_${i}.gro -r NaCl_${i}.gro -p NaCl_US.top -o NaCl_US.tpr -maxwarn 1  # Generate the tpr
    cd ../
done

In [ ]:
%%time
%%bash
mpirun -np 8 gmx_mpi mdrun -s NaCl_US.tpr -deffnm output -pf pullf.xvg -px pullx.xvg -multidir sim_1 sim_2 sim_3 sim_4 sim_5 sim_6 sim_7 sim_8 -ntomp 4

In [ ]:
pullx_data = [np.transpose(np.loadtxt(f'sim_{i + 1}/pullx.xvg', comments=['@', '#'])) for i in range(8)]
time = pullx_data[0][0]
dist_list = [data[1] for data in pullx_data]
plt.figure()
for i in range(8):
    plt.hist(dist_list[i], bins=50, alpha=0.5, label=f'sim_{i}')
plt.xlabel('Ion-pair distance (nm)')
plt.ylabel('Count')
plt.grid()

In [ ]:
import time
import pymbar
from pymbar import timeseries
import random
import scipy.stats

t0 = time.time()

# Step 1: Setting up
K = 8                                       # number of umbrellas
N_max = 5001                                # number of data points in each timeseries of ion-pair distance
kT = 1.381e-23 * 6.022e23 / 1000 * 300      # 1 kT converted to kJ/mol at 300 K
beta_k = np.ones(K) / kT                    # inverse temperature of simulations (in 1/(kJ/mol)) 
d_min, d_max = 0.25, 0.65                   # minimum and maximum of the CV for plotting the FES
nbins = 50                                  # number of bins for 1D PMF
K_k = np.ones(K) * 10000                    # spring constant (in kJ/mol/nm**2) for different simulations
RunBootstrapping = True                     # Whether to perform bootstrapping to estimate the uncertainty of the FES
nboot = 200                                 # number of bootstrap samples
N_k, g_k = np.zeros(K, int), np.zeros(K)    # number of samples and statistical inefficiency of different simulations
d_kn = np.zeros([K, N_max])                 # d_kn[k,n] is the ion-pair distance (in nm) for snapshot n from umbrella simulation k
u_kn = np.zeros([K, N_max])                 # u_kn[k,n] is the reduced potential energy without umbrella restraints of snapshot n of umbrella simulation k

# Step 2: Read in and subsample the timeseries
for k in range(K):
    d_kn[k] = np.transpose(np.loadtxt(f'sim_{k + 1}/pullx.xvg', comments=['@', '#']))[1]
    N_k[k] = len(d_kn[k])
    d_temp = d_kn[k, 0:N_k[k]]
    g_k[k] = timeseries.statisticalInefficiency(d_temp)     
    print(f"Statistical inefficiency of simulation {k}: {g_k[k]:.3f}")
    indices = timeseries.subsampleCorrelatedData(d_temp, g=g_k[k]) # indices of the uncorrelated samples
    
    # Update u_kn and d_kn with uncorrelated samples
    N_k[k] = len(indices)    # At this point, N_k contains the number of uncorrelated samples for each state k                
    u_kn[k, 0:N_k[k]] = u_kn[k, indices]
    d_kn[k, 0:N_k[k]] = d_kn[k, indices]

d0_k = np.array([d_kn[i][0] for i in range(K)])    
N_max = np.max(N_k) # shorten the array size
u_kln = np.zeros([K, K, N_max]) # u_kln[k,l,n] is the reduced potential energy of snapshot n from umbrella simulation k evaluated at umbrella l
u_kn -= u_kn.min()  # shift the minimum of the FES to 0

# Step 3: Bin the data
delta = (d_max - d_min) / float(nbins)
bin_center_i = np.array([d_min + delta / 2 + delta * i for i in range(nbins)])
bin_kn = np.zeros([K, N_max])
for k in range(K):
    for n in range(N_k[k]):
        bin_kn[k, n] = int((d_kn[k,n] - d_min) / delta) # contains the information about which bin the data was assigned to
   

# Step 4: Evaluate reduced energies in all umbrellas
for k in range(K):
    for n in range(N_k[k]):
        # Compute minimum-image ion-pair distance deviation from umbrella center l
        dd = d_kn[k,n] - d0_k  # delta d

        # Compute energy of snapshot n from simulation k in umbrella potential l
        u_kln[k,:,n] = u_kn[k,n] + beta_k[k] * (K_k / 2) * dd ** 2

# Step 5: Compute, output, and plot the FES
mbar = pymbar.MBAR(u_kln, N_k, verbose = False)
results = mbar.computePMF(u_kn, bin_kn, nbins)
f_i = results[0]
df_i = results[1]

if RunBootstrapping is False:
    with open('MBAR_PMF.dat', 'w') as f:
        f.write("# PMF (in units of kT) \n")
        f.write(f"# {'bin':>8s} {'f':>8s} {'df':>8s} \n")
        for i in range(nbins):
           f.write(f"{bin_center_i[i]:>8.3f} {f_i[i]:>8.3f} {df_i[i]:>8.3f} \n")

    plt.figure()
    plt.plot(bin_center_i, f_i)
    plt.xlabel('Ion-pair distance (nm)')
    plt.ylabel('Free energy (kT)')
    plt.grid()
else:
    # Step 6: Perform bootstrapping if wanted
    print('\nStart bootstrapping!')
    print('Number of bootstrap iterations:', nboot)
    u_kln_b = np.zeros([K, K, N_max, nboot], np.float64)  
    u_kn_b = np.zeros([K,N_max, nboot])
    d_kn_b = np.zeros([K,N_max, nboot])

    for j in range(nboot):
        for i in range(K):
            # first randomly select N_k[0] (number of uncorrelated samples) samples from the data of i-th simulation
            # lengths of d_kn_rand and u_nk_rankd = number of uncorrelated samples
            d_kn_rand = random.choices(d_kn[i,:], k=N_k[i])
            u_kn_rand = random.choices(u_kn[i,:], k=N_k[i])
            # then assign values to bootstrapping array
            d_kn_b[i,:N_k[i],j] = d_kn_rand
            u_kn_b[i,:N_k[i],j] = u_kn_rand

    u_kn_b -= u_kn_b.min() # might be a problem if not constant temp (to be modified)

    bin_kn_b = np.zeros([K,N_max,nboot])
    for j in range(nboot):
        for k in range(K):
            for n in range(N_k[k]):
                bin_kn_b[k,n,j] = int((d_kn_b[k,n,j] - d_min) / delta)
                dd_b = d_kn_b[k,n,j] - d0_k
                u_kln_b[k,:,n,j] = u_kn_b[k,n,j] + beta_k[k] * (K_k/2.0) * dd_b**2   

    MBAR_b = []  # A list of object of pymbar.MBAR(u_kln_b[j], N_k, verbose = True)
    bootdata = np.zeros([nboot, nbins])  # bootstrapped data for each value of reaction coordinate (data of f_i)

    for j in range(nboot):
        MBAR_b.append(pymbar.MBAR(u_kln_b[:,:,:,j], N_k, verbose = False, initial_f_k = mbar.f_k))
        # initial_f_k = m_bar.f_k is use the mbar.f_k calculated previously as the initial guess, which can speed up the process (default = None, that is f_k = [0, 0, ...,0])
        # take a lookt a __init__ of pymbar and see how to output f_k data --> just use mbar.f_k !
        results_b = MBAR_b[j].computePMF(u_kn_b[:,:,j], bin_kn_b[:,:,j], nbins)
        bootdata[j,:] = results_b[0]      # results_b[0]: data of f_i 

    # Rule of thumb: It requires about 50-200 bootstrap samples to get a good variance and 1000-5000 bootstrap samples to get a good 95CI 
    confidence = 0.95
    m = np.array([np.mean(bootdata[:, i]) for i in range(nbins)])
    sem = np.array([scipy.stats.sem(bootdata[:, i]) for i in range(nbins)])
    h = sem * scipy.stats.t.ppf((1 + confidence) / 2, len(bootdata[:, i]) - 1)  # CI: from m-h to m + h
            
    with open('MBAR_PMF_bootstrapped.dat', 'w') as f:
        f.write("# PMF (in units of kT) \n")
        f.write(f"# {'bin':>8s} {'f':>8s} {'df':>8s} {'f_i_err':>8s}\n")
        for i in range(nbins):
           f.write(f"{bin_center_i[i]:>8.3f} {m[i]:>8.3f} {df_i[i]:>8.3f} {h[i]:>8.3f} \n")
    
    # One could also use variance for error bars (with ddof=1)
    plt.figure()
    plt.title('95% of confidence level')
    plt.plot(bin_center_i, m)
    plt.fill_between(bin_center_i, m - h, m + h, color='lightgreen')
    plt.xlabel('Ion-pair distance (nm)')
    plt.ylabel('Free energy (kT)')
    plt.grid()

t1 = time.time()
print(f'\nTime elapsed: {t1 - t0:.0f} seconds.')